In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install streamlit
!pip install pyngrok

## .ipynb 파일 코드 통째로 넣음



*   classification_using_image.ipynb
*   classification_using_features.ipynb



In [ ]:
# 밑 코드를 app.py에 넣어서 저장
# 내용은 위의 2개 ipynb 파일과 동일
# 코드 굳이 볼 필요 X (괜히 더 복잡)

%%writefile app.py

import tensorflow as tf
import math
import numpy as np
import json
import cv2 as cv
import random
from datetime import datetime
import os
import torch
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, random_split
import streamlit as st
import torch
from PIL import Image
import json
import transformers

CLASS = ['성충_날개불구바이러스감염증',
        '성충_응애',
        '성충_정상',
        '유충_부저병',
        '유충_석고병',
        '유충_응애',
        '유충_정상']
IM_HEIGHT = 224
IM_WIDTH = 224


def read_data_1_input(json_file):

    temp, hum, co2 = get_data(json_file)

    temp = div_func(temp)
    temp = normalize_resize_concat(temp, resize=True)

    hum = div_func(hum)
    hum = normalize_resize_concat(hum, resize=True)

    co2 = div_func(co2)
    co2 = normalize_resize_concat(co2, resize=True)

    return cv.merge((temp, hum, co2))


def get_data(annot_file):
    with open(annot_file, "r", encoding='UTF-8-SIG') as f:
        data = json.loads(f.read())

    temp = np.array(data["environment"]['in_temperature'], dtype='float')
    humidity = np.array(data["environment"]['in_humidity'], dtype='float')
    co2 = np.array(data["environment"]['in_carbon_monoxide'], dtype='float')

    return temp, humidity, co2

def normalize_resize_concat(x, hist=False, resize=False, merge=False):

    x = cv.normalize(x, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    x = np.array(x, np.uint8)

    if resize:
        x = cv.resize(x, (IM_HEIGHT, IM_WIDTH), cv.INTER_AREA)

    if hist:
        x = cv.equalizeHist(x)

    if merge:
        x = cv.merge((x, x, x))

    return x

def get_prediction(model_file, raw_data):
    data = read_data_1_input(raw_data)
    data = preprocess_image(np_image=data, net_type='efficientnet')
    prediction = model_file.predict(np.expand_dims(data, axis=0), batch_size=1)
    return np.argmax(prediction)

def make_model_efficientnet_1input():
    m1 = tf.keras.applications.EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    gap1 = tf.keras.layers.GlobalAveragePooling2D()(m1.output)
    l3 = tf.keras.layers.Dense(7, activation='softmax')(gap1)
    model = tf.keras.models.Model(inputs=m1.inputs, outputs=l3)
    return model

# 모델 불러오기
def load_model1():
  # MODEL LOADING
    model = make_model_efficientnet_1input()
    net_name = '/content/drive/MyDrive/제4회 AI교육 해커톤/network.hdf5'
    model.load_weights(net_name)
    # model = model.to('cpu')
    return model

def load_model2():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = models.resnet18(pretrained=False)
    num_ftrs = model.fc.in_features
    model.fc = torch.nn.Linear(num_ftrs, 7)
    model_path = '/content/drive/MyDrive/제4회 AI교육 해커톤/bee_classification_resnet.pth'
    model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    model = model.to('cpu')
    model.eval()  # 추론 모드로 전환 (학습용이 아닌 추론용)
    return model

# JSON 파일 처리 함수
def process_json_file(json_data, model):
    prediction = get_prediction(model, json_data)
    prediction = CLASS[prediction]
    st.write(f"Prediction: {prediction}")
    result = f"Processed JSON with {model}"
    return result

# 이미지 처리 함수
def process_image(image, model):
    preprocess = transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
  ])
    input_tensor = preprocess(image)
    input_batch = input_tensor.unsqueeze(0)  # 배치 차원 추가

    # 모델에 입력하여 추론 실행
    with torch.no_grad():  # 그래디언트 계산 비활성화
        output = model(input_batch)

    # 소프트맥스 적용하여 확률 변환
    probabilities = torch.nn.functional.softmax(output[0], dim=0)

    # 클래스 이름 정의 (학습 시 사용한 클래스와 동일하게)

    # 가장 높은 확률의 클래스 예측
    _, predicted_class = torch.max(probabilities, 0)

    return CLASS[predicted_class]

def preprocess_image(np_image, net_type='efficientnet'):
    assert net_type in ['mobilenet', 'resnet50', 'efficientnet'], 'network not in the list'

    m_image = tf.convert_to_tensor(np_image, dtype=tf.float32)

    # if net_type == 'resnet50':
    #     return preprocess_input_resnet50(m_image)

    if net_type == 'efficientnet':
        return m_image

def read_data_1_input(json_file):

    temp, hum, co2 = get_data(json_file)

    temp = div_func(temp)
    temp = normalize_resize_concat(temp, resize=True)

    hum = div_func(hum)
    hum = normalize_resize_concat(hum, resize=True)

    co2 = div_func(co2)
    co2 = normalize_resize_concat(co2, resize=True)

    return cv.merge((temp, hum, co2))


def get_data(annot_file):
    data = json.load(annot_file)

    temp = np.array(data["environment"]['in_temperature'], dtype='float')
    humidity = np.array(data["environment"]['in_humidity'], dtype='float')
    co2 = np.array(data["environment"]['in_carbon_monoxide'], dtype='float')

    return temp, humidity, co2


def div_func(x):
    x = x.squeeze()
    x1 = x[:189]
    for n in range(1, 189):
        x1 = np.vstack((x1, x[n*189:(n+1)*189]))
    return x1

def normalize_resize_concat(x, hist=False, resize=False, merge=False):

    x = cv.normalize(x, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype=cv.CV_32F)
    x = np.array(x, np.uint8)

    if resize:
        x = cv.resize(x, (IM_HEIGHT, IM_WIDTH), cv.INTER_AREA)

    if hist:
        x = cv.equalizeHist(x)

    if merge:
        x = cv.merge((x, x, x))

    return x

def make_model_efficientnet_1input():
    m1 = tf.keras.applications.EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    gap1 = tf.keras.layers.GlobalAveragePooling2D()(m1.output)
    l3 = tf.keras.layers.Dense(7, activation='softmax')(gap1)
    model = tf.keras.models.Model(inputs=m1.inputs, outputs=l3)
    return model

model2 = load_model2()  # 이미지를 처리하는 모델
model2 = model2.to('cpu')

# Streamlit 앱 시작
st.title("JSON 또는 이미지 입력에 따른 모델 실행")

# 파일 업로드 기능
uploaded_file = st.file_uploader("JSON 파일 또는 이미지 파일을 업로드하세요", type=["json", "jpg", "jpeg", "png"])

# llm으로 생성한 질병 설명 텍스트
llm_results = {'성충_날개불구바이러스감염증': '날개불구바이러스감염증은 꿀벌의 체액을 빨아먹는 바로아 응애에 의해 전파되는 바이러스성 질병입니다. 이 질병으로 감염된 꿀벌은 날개 기형을 일으키고 비행 능력을 상실하게 됩니다. 군체 내에서 바이러스가 전염되면 꿀벌의 활력이 떨어지고 벌 개체 수가 급격히 감소하는 등 치명적인 결과를 초래할 수 있습니다. 이를 예방하고 치료하려면 응애를 제거하고 화학적 방법으로 항바이러스 치료를 수행하며 군체 관리를 철저히 해야 합니다.',
                '성충_응애': '성충_응애는 꿀벌의 날개 기형을 일으키는 바이러스성 질병입니다. 이 질병은 바로아 응애가 꿀벌의 체액을 빨아먹으며 바이러스를 전달하는 과정에서 발생합니다. 감염된 꿀벌은 날개 기형을 일으키고 비행 능력을 상실하여 짧은 수명을 갖게 됩니다. 이를 방지하기 위해 응애 제거, 항바이러스 치료, 군체 관리 등을 통해 질병을 관리해야 합니다.',
                '유충_부저병': '유충_부저병은 꿀벌의 유충이 부저에 감염되는 바이러스성 질병입니다. 이 질병은 꿀벌의 유충이 부저에 있는 바이러스에 감염되어 부저가 부식되거나 파괴되는 것을 일으킵니다. 이러한 질병은 꿀벌의 생산을 저해하고, 꿀벌의 생존을 위협합니다. 질병을 예방하고 치료하기 위해서는 꿀벌의 유충을 적절하게 관리하고, 부저를 청결하게 하는 것이 중요합니다.',
                '유충_석고병': '유충_석고병은 꿀벌의 유충이 석고균에 감염된 질병입니다. 이 질병은 유충이 석고균에 의해 파괴되는 경우를 초래하고, 이에 따라 꿀벌의 생산이 저하됩니다. 석고균은 꿀벌의 유충을 죽이는 데 사용되는 화학 물질을 생산하는 데 사용됩니다. 이 질병은 꿀벌의 생산을 저하하고, 꿀벌의 건강을 악화시킬 수 있습니다.',
                '유충_응애': '유충_응애는 꿀벌의 유충이 응애에 의해 감염되는 바이러스성 질병입니다. 이 질병은 꿀벌의 유충이 응애에 기생당하여 바이러스를 전달받아 감염됩니다. 감염된 꿀벌의 유충은 체력이 약해지게 되고, 성충이 되지 못하는 경우가 많습니다. 이 질병은 꿀벌 군체의 활력을 떨어뜨리고, 벌 개체 수를 감소시킬 수 있습니다.'}

m_prediction = None

if uploaded_file is not None:
    file_extension = uploaded_file.name.split('.')[-1].lower()

    if file_extension == "json": # 시계열 데이터 (json)을 입력으로 받을 경우
        model = make_model_efficientnet_1input()
        net_name = '/content/drive/MyDrive/제4회 AI교육 해커톤/network.hdf5'
        model.load_weights(net_name)

        data = read_data_1_input(uploaded_file)
        processed_data = preprocess_image(data, net_type='efficientnet')
        prediction = model.predict(np.expand_dims(processed_data, axis=0), batch_size=1)
        m_prediction = np.argmax(prediction)

        # 결과 출력
        st.write(f'Prediction is class {m_prediction}, => {CLASS[m_prediction]}')


    elif file_extension in ["jpg", "jpeg", "png"]: # 이미지를 입력으로 받을 경우
        try:
            image = Image.open(uploaded_file)
            st.image(image, caption='Uploaded Image', use_column_width=True)
            result = process_image(image, model2)
            m_prediction = CLASS.index(result)
            st.success("이미지 파일이 성공적으로 처리되었습니다.")
            st.write(f'Prediction is class {m_prediction}, => {CLASS[m_prediction]}')
        except Exception as e:
            st.error(f"이미지 파일을 처리하는 중 오류가 발생했습니다: {e}")

    else:
        st.error("지원되지 않는 파일 형식입니다. JSON, JPG, JPEG 또는 PNG 파일을 업로드하세요.")

    # 예측한 결과에 따라 llm 설명 텍스트 출력
    if m_prediction is not None:
        st.title('상세 설명')
        prediction = CLASS[m_prediction]
        if prediction in llm_results:
            st.write(llm_results[prediction])
        else:
            st.write("정상입니다.")

Overwriting app.py


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.169.81.153


In [ ]:
# 이 코드 실행해서 "your url is" 링크 들어가면 key 입력 필요 => 바로 위 코드에 있는 숫자 복붙 (이게 key)
# 런타임 초기화되면 key 바뀜
!streamlit run /content/app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.169.81.153:8501

your url is: https://few-goats-happen.loca.lt
2024-10-13 18:38:57.432 Uncaught exception GET /_stcore/stream (127.0.0.1)
HTTPServerRequest(protocol='http', host='few-goats-happen.loca.lt', method='GET', uri='/_stcore/stream', version='HTTP/1.1', remote_ip='127.0.0.1')
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/tornado/websocket.py", line 937, in _accept_connection
    open_result = handler.open(*handler.open_args, **handler.open_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/streamlit/web/server/browser_websocket_handler.py", line 126, in open
    self._session_id = self._runtime.connect_session(
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/runtime.py", line 384, in connect_session
    session_id = self._session_mgr.connect_session(
  File "/us